In [248]:
import torch
import numpy as np
import random

In [249]:
dtype = torch.float
device = torch.device("cpu")

In [250]:
class NN(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(NN, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H,D_out)
        
    def forward(self,x):
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return (y_pred)
            
        

In [251]:
class NN2(torch.nn.Module):
    def __init__(self, D_in,H, D_out):
        super(NN2,self).__init__()
        self.linear1 = torch.nn.Linear(D_in,H)
        self.middle = torch.nn.Linear(H,H)
        self.linear2 = torch.nn.Linear(H, D_out)
    
    def forward(self, x):
        h_relu  = self.linear1(x).clamp(min = 0)
        for _ in range(random.randint(0,3)):
            h_relu = self.middle(h_relu).clamp(min=0)
        
        return self.linear2(h_relu)

In [228]:
class myRelu(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return input.clamp(min=0)
    
    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input <0] = 0
        return grad_input
        
    

In [252]:
N,D_in, H, D_out = 64,50,100,10

In [223]:
model = torch.nn.Sequential(
    torch.nn.Linear(D_in,H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),

)

In [232]:
model = NN(D_in,H,D_out)

In [253]:
model = NN2(D_in,H,D_out)

In [254]:
loss_fn = torch.nn.MSELoss(size_average=False)


In [255]:
x = torch.randn(N,D_in, dtype=dtype, device=device)
y = torch.randn(N,D_out, dtype=dtype, device=device)

w1 = torch.randn(D_in, H, dtype=dtype, device=device, requires_grad=True)
w2 = torch.randn(H, D_out, dtype=dtype, device=device, requires_grad=True)



In [256]:

lr = 1e-4
optimizer = torch.optim.Adam(model.parameters(),lr=lr)
for t in range(2000):
    y_pred = model(x) 
    loss  = loss_fn(y_pred,y)
    optimizer.zero_grad()
    loss.backward()
    
    optimizer.step()


print(loss.item())

27.683504104614258
